In [1]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object, save_object
import detectron.utils.vis as vis_utils


def checkMkdir(dirname):
    if not osp.isdir(dirname):
        os.makedirs(dirname)


modelType = 'Fashion_Detector' #'HF_Detector' #'COCO_Mask_Detector'
fixedPrec = 0.8

outThrshBoxPath = osp.join('/media/data/chnxi/GOD/threshold_bboxes/', 'prec_at_{}'.format(fixedPrec))
modelOutBoxFolder = osp.join(outThrshBoxPath, modelType)
checkMkdir(modelOutBoxFolder)

if modelType == 'Fashion_Detector':
    cls_thrsh_file = '/home/chnxi/ssd/backfill/data/class_thresholds_at_pr.pkl'
    boxesOnDataset = {
        'furniture_train': '/home/chnxi/ssd/backfill/FashionV2_on_HFTrain/outputs/FashionV2_HFTrain_by_image.json',
        'COCO_trainval': '/home/chnxi/ssd/backfill/FashionV2_on_COCOTrainval/outputs/FashionV2_COCOTrainvalBbox_by_image.json',
        'OpenImage_train': '/home/chnxi/ssd/backfill/FashionV2_on_OpenImage/outputs/FashionV2_OpenImageTrain_Bboxes_by_image.json'
    }
elif modelType == 'HF_Detector':
    cls_thrsh_file = '/home/chnxi/Detectron/backfill/HF_Detector/class_thresholds_at_pr.pkl'
    boxFilePath = '/home/chnxi/Detectron/backfill/HF_Detector/'
    boxesOnDataset = {'FashionV2_train': boxFilePath + 'FashionV2_train/test/FashionV2_train/retinanet/detections.pkl',
    'OpenImage_train': boxFilePath + 'OpenImage_train/test/OpenImage_train/retinanet/detections.pkl', 
    'coco_2014_valminusminival':  boxFilePath + 'coco_trainval/test/coco_2014_valminusminival/retinanet/detections.pkl',
    'coco_2014_train': boxFilePath + 'coco_trainval/test/coco_2014_train/retinanet/detections.pkl'
                     }
    
elif modelType == 'COCO_Mask_Detector':
    cls_thrsh_file = '/home/chnxi/Detectron/backfill/COCO_Mask_Detector/class_thresholds_at_pr.pkl'
    boxFilePath = '/home/chnxi/Detectron/backfill/COCO_Mask_Detector/'
    boxesOnDataset = {'FashionV2_train': boxFilePath + 'FashionV2_train/test/FashionV2_train/generalized_rcnn/detections.pkl',
    'OpenImage_train': boxFilePath + 'OpenImage_train/test/OpenImage_train/generalized_rcnn/detections.pkl', 
    'furniture_train': boxFilePath + 'furniture_train/test/furniture_train/generalized_rcnn/detections.pkl'
                     }

    
print ("modelType = {}===================".format(modelType))
print ("cls_thrsh_file = {}===================".format(cls_thrsh_file))
print ("modelOutBoxFolder = {}===================".format(modelOutBoxFolder))

modelType = Fashion_Detector===================
cls_thrsh_file = /home/chnxi/ssd/backfill/data/class_thresholds_at_pr.pkl===================
modelOutBoxFolder = /media/data/chnxi/GOD/threshold_bboxes/prec_at_0.8/Fashion_Detector===================


In [2]:
cls_thrsh_dict = load_object(cls_thrsh_file)
print (cls_thrsh_dict.keys())
classes = cls_thrsh_dict['classes']

apply_cls_thrsh = cls_thrsh_dict['prec_at'][fixedPrec]

print(len(apply_cls_thrsh))
print(apply_cls_thrsh)
print (len(classes))


dict_keys(['cls_aps', 'classes', 'rec_at', 'prec_at'])
82
[0.921507  0.176479  0.498588  0.425203  0.58903   0.96161   0.388972
 0.996838  0.0868612 0.938948  0.261433  0.632203  0.851458  0.924308
 0.89844   0.746335  0.0464003 0.268311  0.694573  0.762744  0.965019
 0.51947   0.608755  0.7835    0.333918  0.966148  0.850797  0.656509
 0.899731  0.860438  0.893335  0.939373  0.995674  0.562568  0.473903
 0.728581  0.320062  0.890649  0.939149  0.748116  0.976356  0.686515
 0.586246  0.860934  0.724155  0.467066  0.944402  0.503046  0.862419
 0.989524  0.592589  0.640688  0.0402405 0.801507  0.766969  0.894042
 0.434545  0.728986  0.969326  0.462354  0.891931  0.205737  0.0102759
 0.958674  0.887045  0.945219  0.954513  0.0779886 0.900719  0.599443
 0.331076  0.927601  0.61923   0.750555  0.899903  0.857342  0.780312
 0.586498  0.995086  0.690747  0.693917  0.83807  ]
82


In [3]:
if modelType == 'Fashion_Detector':
    for dataset in boxesOnDataset:
        print ("model = {}, dataset = {}".format(modelType, dataset))
        outBoxFile = osp.join(modelOutBoxFolder, dataset + '.json')
        print ("outBoxFile = {}".format(outBoxFile))
        orgBoxFile = boxesOnDataset[dataset]
        print ("loading {}".format(orgBoxFile))
        dets = json.load(open(orgBoxFile,'r'))
        print ("loaded")
        imgNum = len(dets)
        print ("imgNum = {}".format(imgNum))
        validBoxNum = 0
        for ix, entry in enumerate(dets):
            img_boxes = entry['boxes']
            img_boxes_thrshed = []
            for box in img_boxes:
                cid = box['category_id'] -1
                #print ("category: {}, cid = {}, thrsh = {}, score = {}".format(box['category_name'], cid, thresholds[cid], box['score']))
                if box['score'] > apply_cls_thrsh[cid]:
                    img_boxes_thrshed.append(box)
            
            dets[ix]['boxes'] = img_boxes_thrshed
            if ix % 10000 == 0:
                print (dets[ix])
            validBoxNum += len(img_boxes_thrshed)
        print ("Total boxNum = {}".format(validBoxNum))
        print ("saving boxes to {}".format(outBoxFile))
        with open(outBoxFile,'w') as fout:
            json.dump(dets, fout)
        print ("Done!")

model = Fashion_Detector, dataset = furniture_train
outBoxFile = /media/data/chnxi/GOD/threshold_bboxes/prec_at_0.8/Fashion_Detector/furniture_train.json
loading /home/chnxi/ssd/backfill/FashionV2_on_HFTrain/outputs/FashionV2_HFTrain_by_image.json
loaded
imgNum = 79277
{'image_id': '17B0E85E02491E313EF2D908471730BD98FC5D553368E3F7A9F2AB3C25FF05FDB6C24FB7B7A1A7066AE04A2909ADB593E1925D7A6079E81444A3B3E9F1E4A8E3', 'boxes': [], 'image_file_name': '17B0E85E02491E313EF2D908471730BD98FC5D553368E3F7A9F2AB3C25FF05FDB6C24FB7B7A1A7066AE04A2909ADB593E1925D7A6079E81444A3B3E9F1E4A8E3.jpg'}
{'image_id': 'D4EB10F1474573B52E581988DDA7DBC977686DD1A1FFB9585C2CC6F6B7DDDE6D2E01D1CABB3BCE3F2EF0E8193AFA61FF30E067A3347427EF26D4D99610E9D593', 'boxes': [], 'image_file_name': 'D4EB10F1474573B52E581988DDA7DBC977686DD1A1FFB9585C2CC6F6B7DDDE6D2E01D1CABB3BCE3F2EF0E8193AFA61FF30E067A3347427EF26D4D99610E9D593.jpg'}
{'image_id': '020EC895A1C926210AD45E2E213B6AA0F1F676EDEE19A3E64A4D6B6FF5C1C38BF1D92AB491C7D1A162A5F1DA46

{'image_id': 'de8d1646d4bdb17f', 'boxes': [], 'image_file_name': 'de8d1646d4bdb17f.jpg'}
{'image_id': 'c45f99039bb47724', 'boxes': [], 'image_file_name': 'c45f99039bb47724.jpg'}
{'image_id': 'd64fcc898b0a7238', 'boxes': [], 'image_file_name': 'd64fcc898b0a7238.jpg'}
{'image_id': 'd1e3692707be18d3', 'boxes': [], 'image_file_name': 'd1e3692707be18d3.jpg'}
{'image_id': 'dc6149ff8883e334', 'boxes': [], 'image_file_name': 'dc6149ff8883e334.jpg'}
{'image_id': 'd7a0b91d2d83d457', 'boxes': [{'score': 0.228, 'category_id': 2, 'bbox': [506.49920000000003, 414.4128, 657.8248, 544.1536], 'category_name': 'accessories-glasses'}], 'image_file_name': 'd7a0b91d2d83d457.jpg'}
{'image_id': 'd3f9e43b8e22a0aa', 'boxes': [], 'image_file_name': 'd3f9e43b8e22a0aa.jpg'}
{'image_id': 'c07945f8b43784de', 'boxes': [], 'image_file_name': 'c07945f8b43784de.jpg'}
{'image_id': 'cc2a5e9c847a5e5d', 'boxes': [], 'image_file_name': 'cc2a5e9c847a5e5d.jpg'}
{'image_id': 'd287abb2e9806ca4', 'boxes': [], 'image_file_name': 

{'image_id': '7338df18c37f536c', 'boxes': [], 'image_file_name': '7338df18c37f536c.jpg'}
{'image_id': '74aca51a6ccfa698', 'boxes': [], 'image_file_name': '74aca51a6ccfa698.jpg'}
{'image_id': '6d27d05931c877d0', 'boxes': [{'score': 0.7318, 'category_id': 2, 'bbox': [617.7792, 122.54650000000001, 726.9376, 172.07199999999997], 'category_name': 'accessories-glasses'}], 'image_file_name': '6d27d05931c877d0.jpg'}
{'image_id': '83304ecc9d461c56', 'boxes': [{'score': 0.7552, 'category_id': 42, 'bbox': [476.2624, 243.9936, 641.536, 452.1984], 'category_name': 'outer_wear-jacket'}], 'image_file_name': '83304ecc9d461c56.jpg'}
{'image_id': '6b4b25e060c684b6', 'boxes': [{'score': 0.9836, 'category_id': 5, 'bbox': [171.8016, 430.2848, 572.16, 955.392], 'category_name': 'accessories-scarf'}], 'image_file_name': '6b4b25e060c684b6.jpg'}
{'image_id': '70dac008e569c91e', 'boxes': [{'score': 0.9943, 'category_id': 4, 'bbox': [151.6544, 95.2835, 560.3328, 477.308], 'category_name': 'accessories-hat'}], 'i

{'image_id': '02615399285a531a', 'boxes': [], 'image_file_name': '02615399285a531a.jpg'}
{'image_id': '065cc26b9cbbe8a4', 'boxes': [], 'image_file_name': '065cc26b9cbbe8a4.jpg'}
{'image_id': '002d9c1d2addb068', 'boxes': [], 'image_file_name': '002d9c1d2addb068.jpg'}
{'image_id': '039e2a6563f0a9d0', 'boxes': [{'score': 0.6024, 'category_id': 4, 'bbox': [67.82976, 64.38633, 221.2864, 189.2124], 'category_name': 'accessories-hat'}, {'score': 0.7493, 'category_id': 19, 'bbox': [656.0768, 465.7536, 864.768, 757.3161], 'category_name': 'bottom_wear-jeans'}, {'score': 0.8989, 'category_id': 23, 'bbox': [665.4976, 631.8276000000001, 953.0368, 825.2384999999999], 'category_name': 'bottom_wear-skirt'}, {'score': 0.4596, 'category_id': 71, 'bbox': [56.66816, 158.0502, 230.8096, 462.11490000000003], 'category_name': 'top_wear-short-sleeve-button'}], 'image_file_name': '039e2a6563f0a9d0.jpg'}
{'image_id': '0639d0d51bf59193', 'boxes': [], 'image_file_name': '0639d0d51bf59193.jpg'}
{'image_id': '08f0

In [6]:
if modelType in ['HF_Detector', 'COCO_Mask_Detector']:
    for dataset in boxesOnDataset:
        print ("model = {}, dataset = {}".format(modelType, dataset))
        outBoxFile = osp.join(modelOutBoxFolder, dataset + '.pkl')
        print ("outBoxFile = {}".format(outBoxFile))
        orgBoxFile = boxesOnDataset[dataset]
        print ("loading {}".format(orgBoxFile))
        dets = load_object(orgBoxFile)
        print ("loaded")
        all_boxes = dets['all_boxes'][1:] # all_boxes[0] is for background
        classNum = len(all_boxes)
        imgNum = len(all_boxes[0])
        print ("classNum = {}, imgNum = {}".format(classNum, imgNum))
        dataset_thrsh_boxes = [] # in dataset_thrsh_boxes[imgid][cid] format
        validBoxNum = 0

        for imgid in range(imgNum):
            keep_img_boxes = []
            for cid in range(classNum):
                img_cls_boxes = all_boxes[cid][imgid]
                keep_img_cls_boxes = [bbox for bbox in img_cls_boxes if bbox[-1] > apply_cls_thrsh[cid]]
                validBoxNum += len(keep_img_cls_boxes)
                keep_img_boxes.append(keep_img_cls_boxes)
            dataset_thrsh_boxes.append(keep_img_boxes)

        print ("Total boxNum = {}".format(validBoxNum))
        print ("saving boxes to {}".format(outBoxFile))
        save_object(dataset_thrsh_boxes, outBoxFile)
        print ("Done!")


model = HF_Detector, dataset = FashionV2_train
outBoxFile = /media/data/chnxi/GOD/threshold_bboxes/prec_at_0.8/HF_Detector/FashionV2_train.pkl
loading /home/chnxi/Detectron/backfill/HF_Detector/FashionV2_train/test/FashionV2_train/retinanet/detections.pkl
loaded
classNum = 58, imgNum = 160916
Total boxNum = 2628
saving boxes to /media/data/chnxi/GOD/threshold_bboxes/prec_at_0.8/HF_Detector/FashionV2_train.pkl
Done!
model = HF_Detector, dataset = OpenImage_train
outBoxFile = /media/data/chnxi/GOD/threshold_bboxes/prec_at_0.8/HF_Detector/OpenImage_train.pkl
loading /home/chnxi/Detectron/backfill/HF_Detector/OpenImage_train/test/OpenImage_train/retinanet/detections.pkl
loaded
classNum = 58, imgNum = 1743042
Total boxNum = 483956
saving boxes to /media/data/chnxi/GOD/threshold_bboxes/prec_at_0.8/HF_Detector/OpenImage_train.pkl
Done!
model = HF_Detector, dataset = coco_2014_valminusminival
outBoxFile = /media/data/chnxi/GOD/threshold_bboxes/prec_at_0.8/HF_Detector/coco_2014_valminusminival.